In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os
import cv2 
from google.colab.patches import cv2_imshow
from PIL import Image
import sys
import math
import random
from skimage.transform import rotate, AffineTransform
from skimage.util import random_noise
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, ConcatDataset
import random
import tensorflow as tf
from tensorflow import keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Preparation

For the lack of RAM, we are only reading 1000 images per class. Total 10k

In [3]:
# This class reads the data for each dataset
class readDataset:
    def __init__(self, num_classes, path):
        self.height = 200
        self.width = 200
        self.channels = 3
        self.path = path
        self.num_classes = num_classes

    def Animal10(self):
        class_names = ['cane', 'cavallo', 'elefante',
                       'farfalla', 'gallina', 'gatto',
                       'mucca', 'pecora', 'ragno', 'scoiattolo']
        # list which contains data
        data = []
        data_labels = []
        # read path names of dataset
        for i in range(self.num_classes) :
            num_data_per_class = 0
            path = self.path +class_names[i] 
            data_names = os.listdir(path)
            print("\n class: ",class_names[i])
            print("number of data: ",len(data_names))
            # read whole data
            for j in range(len(data_names)):
                path_to_image = path + "/" + data_names[j]
                image = cv2.imread(path_to_image)
                image = cv2.resize(image, (self.height, self.width))
                data.append(np.array(image))
                data_labels.append(i)
                # filter over number of images per class
                if num_data_per_class ==1000:
                    break
                num_data_per_class += 1

        # return two lists
        data = np.array(data)
        # encode the labels
        data_labels = tf.keras.utils.to_categorical(np.array(data_labels), self.num_classes)

        return data, data_labels


In [4]:
dataset_path = "/content/drive/MyDrive/projectDataset/animal10/raw-img/"
animal10 = readDataset(10, dataset_path)
data, data_labels = animal10.Animal10()


 class:  cane
number of data:  5050

 class:  cavallo
number of data:  2630

 class:  elefante
number of data:  1454

 class:  farfalla
number of data:  2123

 class:  gallina
number of data:  3098

 class:  gatto
number of data:  1689

 class:  mucca
number of data:  1898

 class:  pecora
number of data:  1820

 class:  ragno
number of data:  4849

 class:  scoiattolo
number of data:  1862


In [7]:
# this cell is saving the train and test data in to the drive
path_save = '/content/drive/MyDrive/projectDataset/animal10/numpyFiles/'
np.save(path_save+'data.npy', data)
np.save(path_save+'data_labels.npy',data_labels)

In [ ]:
# this cell is loading the numpies from the drive
# uncomment in case

# path_save = '/content/drive/MyDrive/projectDataset/dogsAndCats/dataset/numpyFiles/'
# train_data = np.load(path_save+'catDogs_train_data.npy')
# train_labels = np.load(path_save+'catDogs_train_labels.npy')
# test_data = np.load(path_save+'catDogs_test_data.npy')
# test_labels = np.load(path_save+'catDogs_test_labels.npy')

In [ ]:
# normalizing the data
# train_data = train_data/255.0
# test_data = test_data/255.0

In [8]:
# split data to train and test
train_data, test_data, train_labels, test_labels = train_test_split(data, 
                                                                 data_labels, 
                                                                 test_size=0.2)
# split train to train and val
train_data, val_data, train_labels, val_label = train_test_split(train_data, 
                                                                 train_labels, 
                                                                 test_size=0.2)

#Data Augmentation in Pytorch

In [9]:
# create dataloader pytorch
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, TensorDataset
# convert numpy to tensors
train_data = torch.from_numpy(train_data.astype(np.float32))
test_data = torch.from_numpy(test_data.astype(np.float32))
val_data = torch.from_numpy(val_data.astype(np.float32))
train_labels = torch.from_numpy(train_labels.astype(np.float32))
test_labels = torch.from_numpy(test_labels.astype(np.float32))
val_labels = torch.from_numpy(val_label.astype(np.float32))
# create tensor dataset
train_dataset = TensorDataset(train_data, train_labels)
val_dataset = TensorDataset(val_data, val_labels)
test_dataset = TensorDataset(test_data, test_labels)

In [10]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(degrees=45),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15)),
    # transforms.RandomResizedCrop(size=(224, 224), scale=(0.5, 1.0)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
# this list includes k random number within the range of train length
train_random_list = random.sample(range(0, len(train_data)), round(0.5 * len(train_data)))
val_random_list = random.sample(range(0, len(val_data)), round(0.5 * len(val_data)))

In [12]:
# Apply transformations to each sample in the dataset
transformed_train = []
cnt = 0
for sample, label in train_dataset:
    if cnt in train_random_list:
      transformed_sample = transform(sample.permute(2, 0, 1))
      # print(transformed_sample.shape)
      transformed_train.append((transformed_sample.permute(1, 2, 0), label))
    cnt += 1

In [13]:
# concatenate the two datasets
augmented_train_dataset = ConcatDataset([train_dataset, transformed_train])

In [14]:
# Apply transformations to each sample in the val dataset
transformed_val = []
cnt = 0
for sample, label in val_dataset:
    if cnt in val_random_list:
      transformed_sample = transform(sample.permute(2, 0, 1))
      # print(transformed_sample.shape)
      transformed_val.append((transformed_sample.permute(1, 2, 0), label))
    cnt += 1

In [15]:
# concatenate the two datasets
augmented_val_dataset = ConcatDataset([val_dataset, transformed_val])

In [16]:
# create dataloader 
train_loader = DataLoader(augmented_train_dataset, batch_size=128,
                          shuffle=True, drop_last=True)
val_loader = DataLoader(augmented_val_dataset, batch_size=128, 
                        shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=128, 
                         shuffle=True, drop_last=True)

In [17]:
for train, label in train_loader:
    print(train.shape)
    break

torch.Size([128, 200, 200, 3])
